
### 1. Monte-Carlo （蒙特卡洛方法）

#### 1. Goal （目标）
**核心目标**：在给定策略 π 的情况下，直接从与环境的**完整交互经历**中学习该策略的状态价值函数 Vπ(s) 或动作价值函数 $Q_{\pi}(s, a)$。
**关键思想**：通过“经验”而非“模型”来学习。它通过反复运行多个回合episode，然后用每个状态或状态-动作对所获得的**实际回报**的平均值来估计其价值。

**简单总结**：用“事后诸葛亮”的方式，通过计算历史数据的平均回报来评估策略。

---

#### 2. First-Visit MC Policy Evaluation （首次访问型MC策略评估）

**定义**：在一个episode中，对于某个状态 s，只使用**第一次**访问到它时得到的回报来计算平均值。

**算法步骤**：
1.  **输入**：待评估的策略 π。
2.  **循环**：使用策略 π 生成大量完整的回合经历episode。
3.  **处理每个回合**：
    *   从回合开始到结束，记录每个状态和其后的累积回报。
    *   对于这个episode中出现的每一个状态 s：
        *   如果这是该状态 s 在本episode中**第一次**出现，则：
            *   记录从这次出现开始直到回合结束所获得的**实际回报 G**。
            *   将这个 G 值加入到状态 s 的“回报总和”中:$S(s) \leftarrow S(s)+ G_t$
            *   将状态 s 的“访问计数”加 1:$N(s) \leftarrow N(s)+1$
4.  **评估价值**：对于所有状态 s，其价值估计为 $V(s) = S(s) / N(s)$

**优点**：对每个回合中的每个状态只计算一次，保证每个数据点独立同分布。
**缺点**：数据利用率较低，如果一个状态在一个回合中出现多次，只使用第一次的数据。

---

#### 3. Every-Visit MC Policy Evaluation （每次访问型MC策略评估）

**定义**：在一个回合中，对于某个状态 s，**每一次**访问到它时得到的回报都会被用来计算平均值。

**算法步骤**：
与“首次访问型”基本相同，唯一的区别在于第3步：
*   对于这个回合中出现的每一个状态 s：
    *   **每当**状态 s 出现时，都记录从**这次出现**开始直到回合结束所获得的**实际回报 G**。
    *   将这个 G 值加入到状态 s 的“回报总和”中:$S(s) \leftarrow S(s)+ G_t$
    *   将状态 s 的“访问计数”加 1:$N(s) \leftarrow N(s)+1$

**优点**：数据利用率高，能更快地获得更多样本。
**缺点**：同一个回合内对同一状态的多次访问得到的数据不是完全独立的，理论上分析更复杂，但在实践中通常表现良好。

---

#### 4. Incremental Mean & Incremental Monte-Carlo Updates （增量式均值与增量式MC更新）

**核心思想**：无需存储所有历史回报再求平均，而是用一种**在线更新**的方式，每获得一个新数据就立即更新估计值。

**1. 增量式均值公式**：
假设我们已有 n-1 个样本的平均值 $m_{n-1}$，当获得第 n 个样本 $x_n$ 时，新的平均值 $m_n$ 为：
$m_n = m_{n-1} + (1/n) * (x_n - m_{n-1})$
这个公式让我们无需记住所有历史数据，只需记住之前的均值和总次数即可。

**2. 增量式蒙特卡洛更新**：
将增量式均值的思路应用到价值函数的更新中。
*   将状态 s 的价值 V(s) 看作均值。
*   将一次经历中计算出的实际回报 G 看作新样本 $x_n$。

**更新公式**：
$V(S_t) ← V(S_t) + α * [ G_t - V(S_t) ]$
*   $V(S_t)$：在时间 t 状态 S 的价值估计。
*   $G_t$：从时间 t 开始到回合结束的实际回报。
*   $α$：学习率或步长。在标准的增量均值中，$α = 1/N(s)$。但也可以使用一个固定的常数学习率，以便更好地适应非平稳环境。

**优势**：
*   **内存效率高**：无需存储所有历史回报。
*   **在线学习**：每个回合结束后可以立即更新，甚至可以在回合中进行（如TD方法）。
*   **适应性强**：使用固定学习率可以应对环境动态变化的情况。

好的，这是关于时序差分学习（Temporal-Difference Learning）的笔记内容，按照您的提纲进行填充。

### 2. Temporal-Difference （时序差分学习）

#### 1. Goal （目标）
**核心目标**：像蒙特卡洛方法一样，直接从**经验**中学习策略的价值函数，但**无需等待回合结束**，能够进行**在线、增量式**的更新。

**关键思想**：结合了**蒙特卡洛**的采样思想和**动态规划**的**自举**思想。
- **采样**：通过与环境交互来估计期望值（而非使用完整的模型）。
- **自举**：利用当前的估计值来更新另一个估计值。

**简单总结**：通过“猜一猜，然后修正猜测”的方式学习。它猜测后续状态的价值，并立即用实际获得的奖励和这个猜测来更新当前的价值估计。

---

#### 2. TD(0) / One-Step TD

**定义**：TD(0) 是最简单的时序差分学习方法。它只向前看一步，然后立即更新当前的价值估计。

**核心公式（对于状态价值）**：
`V(Sₜ) ← V(Sₜ) + α * [ Rₜ₊₁ + γ * V(Sₜ₊₁) - V(Sₜ) ]`

**公式解读**：
- `V(Sₜ)`：在时间 t 的状态 S 的旧价值估计。
- `α`：学习率，控制更新的步长。
- `Rₜ₊₁`：在状态 Sₜ 执行动作后，**立即获得的实际奖励**。
- `γ`：折扣因子。
- `V(Sₜ₊₁)`：在状态 Sₜ 执行动作后，**转移到的下一个状态 Sₜ₊₁ 的当前价值估计**。
- `[ Rₜ₊₁ + γ * V(Sₜ₊₁) ]`：这被称为 **TD 目标**。它是对真实回报 `Gₜ` 的一个**估计**。
- `[ TD目标 - V(Sₜ) ]`：这被称为 **TD 误差**。它衡量了当前估计与更好估计（TD目标）之间的差异。

**通俗理解**：当我从状态 Sₜ 走到 Sₜ₊₁ 时，我立刻获得了奖励 Rₜ₊₁，并且看到了 Sₜ₊₁ 的价值。我发现 `Rₜ₊₁ + γV(Sₜ₊₁)` 这个“新证据”比我之前对 Sₜ 的估计 `V(Sₜ)` 要好。于是，我将我的估计朝着这个新证据的方向调整一小步（由 α 控制）。

---

#### 3. TD v.s. MC （时序差分与蒙特卡洛的对比）

这是一个核心的权衡，主要体现在两个方面：

##### a) Bias/Variance Trade-Off （偏差/方差权衡）

- **蒙特卡洛**：
    - **高方差，零偏差**。蒙特卡洛的回报 `Gₜ = Rₜ₊₁ + γRₜ₊₂ + ... + γᵀR_T` 依赖于从当前状态到回合结束的**一整条随机路径**，包含了所有后续状态转移和奖励的随机性，因此**方差非常高**。
    - 但是，它是真实回报 `Gₜ` 的**无偏估计**，因为它是直接对目标值进行采样平均。

- **时序差分**：
    - **低方差，有偏差**。TD目标 `Rₜ₊₁ + γV(Sₜ₊₁)` 只依赖于**单一步骤**的随机性（一次状态转移和奖励），因此**方差大大降低**。
    - 但是，它**是有偏的**，因为它的目标是基于**当前有误差的价值估计 `V(Sₜ₊₁)`** 来构建的。如果 `V(Sₜ₊₁)` 不准确，TD目标也就不准确。

**结论**：TD方法通过引入一点偏差，换取了方差的显著降低，这通常使得其学习过程比MC更**稳定、更高效**。

##### b) Certainty Equivalence （确定等价性）

这是一个理解两者差异的深刻视角。

- **蒙特卡洛**：实现的是 **“样本等价性”**。它直接将经验轨迹上观察到的回报进行平均。可以看作是**用经验数据的经验分布**来解决强化学习问题。

- **时序差分**：实现的是 **“确定等价性”**。它通过经验数据学习后，会得到一个模型（价值函数 V(s)）。当用它进行预测时，它表现得**仿佛环境动态模型（转移概率 P 和奖励函数 R）是确定的**，并且就等于它从数据中学到的那个模型。它把学到的价值函数 V 当作是“真实”的，并基于此进行自举更新。

**直观比喻**：
- **MC**：像一个严谨的科学家，每次都要做完整个实验（一个回合）才分析数据，结论可靠但慢。
- **TD**：像一个在比赛中学习的棋手，每走一步就根据局面（新状态的价值）反思上一步的好坏，学得快，但可能因为对局面判断不准而反思错了。

**其他实际区别**：
- **在线性**：TD 可以在每一步后在线学习；MC 必须等到回合结束。
- **对未完结轨迹的处理**：TD 可以处理不完整的轨迹（没有终止状态的情况），MC 不能。

好的，这是对你笔记框架的填充，内容简明并注重数学表达。

---

### 1. Average n-step Returns

**核心思想**：结合不同步长的回报估计，以减少单一n步回报的方差或偏差。

- **n-step Return**：
  \(
  G_t^{(n)} = R_{t+1} + \gamma R_{t+2} + \ldots + \gamma^{n-1} R_{t+n} + \gamma^n V(S_{t+n})
  \)
  它混合了**前n步的真实奖励**和**第n步的状态价值估计**。

- **Average n-step Return**：
  一种直接的方式是将不同n的回报进行平均，例如：
  \(
  \frac{1}{N} \sum_{n=1}^{N} G_t^{(n)}
  \)
  这启发了更优雅的 **λ-return**。

---

### 2. λ-Return

**核心思想**：使用指数加权平均来混合所有可能的n-step returns。

- **λ-Return (\(G_t^\lambda\))**：
  \(
  G_t^\lambda = (1 - \lambda) \sum_{n=1}^{\infty} \lambda^{n-1} G_t^{(n)}
  \)
  对于有限幕情况，最大步长为 \(T-t-1\)：
  \(
  G_t^\lambda = (1 - \lambda) \sum_{n=1}^{T-t-1} \lambda^{n-1} G_t^{(n)} + \lambda^{T-t-1} G_t
  \)

- **权重解释**：
  - \( (1-\lambda) \)：归一化因子，确保权重和为1。
  - \( \lambda^{n-1} \)：赋予n-step return的权重，随n增大而**指数衰减**。
  - \(\lambda=0\)：退化为TD(0)（1-step TD）。
  - \(\lambda=1\)：退化为蒙特卡洛方法。

---

### 3. TD(λ) Weighting Function

**核心思想**：从λ-return的公式中，可以看到其赋予不同n-step return的权重遵循一个**几何分布（指数衰减）**。

- **权重函数**：第n步回报 \(G_t^{(n)}\) 的权重是 \( (1-\lambda)\lambda^{n-1} \)。
- **意义**：这提供了一个平滑的过渡，近期回报（小n）权重高，远期回报（大n）权重低，由λ控制衰减速度。

---

### 4. Eligibility Traces

**核心思想**：引入一个短期记忆向量（**迹向量** `e(s)`），来记录每个状态被访问的“痕迹”，从而实现高效的在线、增量学习。

- **迹的更新**：
  - **累积迹**：
    \(
    \begin{aligned}
    E_0(s) &= 0 \\
    E_t(s) &= \gamma \lambda E_{t-1}(s) + \mathbf{1}(S_t = s)
    \end{aligned}
    \)
    其中 \(\mathbf{1}(S_t = s)\) 是指示函数，当\(S_t = s\)时为1，否则为0。

- **作用**：迹 \(E_t(s)\) 量化了状态`s`在近期被访问的**频率**和**时间远近**。同时出现（高频率）和刚刚出现（时间近）的状态，其迹值更高。

---

### 5. Forward View and Backward View

这是理解TD(λ)的两个等价视角。

#### Forward View (前向视角)

- **定义**：在时刻t，**向前看**，等待并观察到直到幕结束的所有数据，然后计算λ-return \(G_t^\lambda\)，最后用它来更新 \(V(S_t)\)。
- **更新**：\( V(S_t) \leftarrow V(S_t) + \alpha [G_t^\lambda - V(S_t)] \)
- **特点**：**理论性强**，但**非因果**、**非在线**（必须等到幕结束才能更新）。

#### Backward View (后向视角)

- **定义**：在每一步t，**向后看**，根据当前TD Error和每个状态的资格迹，**同时更新所有相关状态**的价值函数。
- **TD Error**：\( \delta_t = R_{t+1} + \gamma V(S_{t+1}) - V(S_t) \)
- **更新**：对于所有状态 \(s \in \mathcal{S}\)，
  \( \Delta V(s) = \alpha \delta_t E_t(s) \)
- **特点**：**在线**、**增量式**、**计算高效**。

#### 等价性

- **TD(λ)定理**：对于离线学习，前向视角和后向视角的**总更新量是等价的**。
- **意义**：这使得我们可以使用计算高效的后向视角算法，来实现理论上前向视角的λ-return效果。我们通常所说的TD(λ)就是指其后向视角的实现。